In [94]:
import pandas as pd
import csv
import torch
import numpy as np
from torchtext import data
import random
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [33]:
tmp = pd.read_csv('standardized.csv')

In [64]:
TEXT = data.Field(
    sequential=True,
    fix_length=500,
    tokenize='spacy',
    pad_first=True,
    lower=True
)

LABEL = data.LabelField(dtype = torch.float,
                        use_vocab=False, 
                        sequential=False,
                        is_target=True)


full = data.TabularDataset('standardized.csv', 'csv', skip_header=True,
        fields=[
            ('docket', None),
            ('outcome', None),
            ('facts', TEXT),
            ('conclusion', None),
            ('target', LABEL)
        ])

TEXT.build_vocab(
    full,
    max_size=20000,
    min_freq=10,
    vectors=None
)

LABEL.build_vocab(full)

In [65]:
print(vars(full.examples[0]))

{'facts': ['  ', 'the', 'subject', 'property', 'consists', 'of', 'a', 'one', '-', 'story', 'commercial', 'building', 'of', 'masonry', 'exterior', 'construction', 'that', 'was', 'built', 'in', '1995', '.', ' ', 'the', 'docket', 'no', ':', '12', '-', '00035.001-c-2', '   ', '2', 'of', '5', 'building', 'contains', '6,040', 'square', 'feet', 'of', 'building', 'area', 'with', 'a', '344', 'square', 'foot', 'canopy', '.', ' ', 'the', 'building', 'is', 'situated', 'on', '33,000', 'square', 'feet', 'of', 'land', 'area', '.', ' ', 'the', 'subject', 'property', 'is', 'a', 'class', 'a', 'office', 'building', 'used', 'as', 'a', 'branch', 'banking', 'facility', '.', ' ', 'the', 'subject', 'property', 'is', 'located', 'in', 'champaign', 'township', ',', 'champaign', 'county', ',', 'illinois', '    ', 'the', 'appellant', 'contends', 'overvaluation', 'as', 'the', 'basis', 'of', 'the', 'appeal', '.', ' ', 'in', 'support', 'of', 'this', 'argument', ',', 'the', 'appellant', 'submitted', 'limited', 'inform

In [88]:
SEED= 189053

train_data, test_data, valid_data = full.split(split_ratio=[0.7, 0.15, 0.15], random_state = random.seed(SEED))

In [90]:
print(f'Number of training examples: {len(train_data)}')
print(f'Number of validation examples: {len(valid_data)}')
print(f'Number of testing examples: {len(test_data)}')

Number of training examples: 31921
Number of validation examples: 6840
Number of testing examples: 6841


In [91]:
print(f"Unique tokens in TEXT vocabulary: {len(TEXT.vocab)}")
print(f"Unique tokens in LABEL vocabulary: {len(LABEL.vocab)}")

Unique tokens in TEXT vocabulary: 17249
Unique tokens in LABEL vocabulary: 2


In [92]:
BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE)

In [105]:
def binary_accuracy(preds, y):
    """
    Return accuracy per batch
    """
    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

class WordEmbAvg(nn.Module):
    def __init__(self, input_dim, embedding_dim, hidden_dim, output_dim, pad_idx):
        
        super().__init__()
        
        # Define an embedding layer, a couple of linear layers, and 
        # the ReLU non-linearity.

        ##YOUR CODE HERE##
        self.embedding = nn.Embedding(input_dim, embedding_dim, padding_idx = pad_idx)  
        self.linear1 = nn.Linear(embedding_dim, hidden_dim)
        self.linear2 = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()
                                 
        
    def forward(self, text):
        embeddings = self.embedding(text)
        embeddings_avg = embeddings.mean(0)
        output = self.linear1(embeddings_avg)
        final = self.linear2(self.relu(output))
        return final
    
class Training_module( ):

    def __init__(self, model):
        self.model = model
        self.loss_fn = nn.BCEWithLogitsLoss()
       
       ##YOUR CODE HERE##
       # Choose an optimizer. optim.Adam is a popular choice
        self.optimizer = optim.Adam(model.parameters(), lr=0.001)
        self.model.train()

    
    def train_epoch(self, iterator):
        '''
        Train the model for one epoch. For this repeat the following, 
        going through all training examples.
        1. Get the next batch of inputs from the iterator.
        2. Determine the predictions using a forward pass.
        3. Compute the loss.
        4. Compute gradients using a backward pass.
        5. Execute one step of the optimizer to update the model paramters.
        '''
        epoch_loss = 0
        epoch_acc = 0
    
        for batch in iterator:
          # batch.facts has the texts and batch.target has the labels.          
            self.optimizer.zero_grad()
            
            ##YOUR CODE HERE##
            predictions = self.model(batch.facts).squeeze(1)
            loss = self.loss_fn(predictions, batch.target)                      
            accuracy = binary_accuracy(predictions, batch.target) 
                   
            loss.backward()
            self.optimizer.step()
                        
            #print(accuracy.item())
            epoch_loss += loss.item()
            epoch_acc += accuracy.item()
            
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    def train_model(self, train_iterator, dev_iterator):
        """
        Train the model for multiple epochs, and after each evaluate on the
        development set.  Return the best performing model.
        """  
        dev_accs = [0.]
        for epoch in range(5):
            self.train_epoch(train_iterator)
            dev_acc = self.evaluate(dev_iterator)
            print(f"Epoch {epoch}: Dev Accuracy: {dev_acc[1]} Dev Loss:{dev_acc[0]}")
            if dev_acc[1] > max(dev_accs):
                best_model = copy.deepcopy(self)
            dev_accs.append(dev_acc[1])
        return best_model.model
                
    def evaluate(self, iterator):
        '''
        Evaluate the performance of the model on the given examples.
        '''
        epoch_loss = 0
        epoch_acc = 0
        
        self.model.eval()
    
        with torch.no_grad():
            print(iterator)
            for batch in iterator:
                print(batch)
                print(self.model(batch.facts))
                predictions = self.model(batch.facts).squeeze(1)
                loss = self.loss_fn(predictions, batch.target)                      
                accuracy = binary_accuracy(predictions, batch.target) 

                epoch_loss += loss.item()
                epoch_acc += accuracy.item()
        
        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
INPUT_DIM = len(TEXT.vocab)
#You can try many different embedding dimensions. Common values are 20, 32, 64, 100, 128, 512
EMBEDDING_DIM = 100
HIDDEN_DIM = 50
OUTPUT_DIM = 1
#Get the index of the pad token using the stoi function
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]


model = WordEmbAvg(INPUT_DIM, EMBEDDING_DIM, HIDDEN_DIM, OUTPUT_DIM, PAD_IDX)




In [106]:
model = model.to(device)
tm = Training_module(model)

#Training the model
best_model = tm.train_model(train_iterator, valid_iterator)

TypeError: '<' not supported between instances of 'Example' and 'Example'

In [ ]:
tm.model = best_model
test_loss, test_acc = tm.evaluate(test_iterator)
print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')